#Import packages & data

##Packages

In [1]:
!pip install tabulate
!pip install netCDF4
!pip install cartopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 24.6 MB/s eta 0:00:00


In [2]:
import requests
import numpy as np
from geopy.geocoders import Nominatim
from tabulate import tabulate
import netCDF4 as nc
import os
import urllib.request
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.colors as colors
import cartopy.feature as cfeature

import xarray as xr

In [3]:
#test = xr.open_dataset(r"C:\Users\sanne\OneDrive\Documents\Study\Master\Guided Research\Data\Oelsmann\Vertical_land_motion_preliminary_point_wise_trends.nc")

##HE data

In [4]:
# Run just one time in the begin to import the needed data

# here we will import the needed file from the SurfDrive

url = "https://surfdrive.surf.nl/files/index.php/s/WSGdJSZW1IEOGgK/download"
filename = "all_as_numpy.zip"
# ijs data geladen
response = requests.get(url, allow_redirects=True)

if response.status_code == 200:
    with open(filename, "wb") as f:
        f.write(response.content)
else:
    print("Failed to download", url)

Failed to download https://surfdrive.surf.nl/files/index.php/s/WSGdJSZW1IEOGgK/download


In [5]:
#  unzip the numpy array's

!unzip -o all_as_numpy.zip -d .  # The point at the end means that unzip will replace the file if already exist

unzip:  cannot find or open all_as_numpy.zip, all_as_numpy.zip.zip or all_as_numpy.zip.ZIP.


In [6]:
# making a list of the numpy arrays that we have and a list of the names
file_names = ['Antarctica.npy', 'Glaciers.npy', 'Greenland.npy',   'Land_low_confidence_SSP585_2100_quantiles50.npy'  ,  'Land_low_confidence_SSP585_2100_quantiles95.npy'  ,
              'Land_low_confidence_SSP585_2300_quantiles50.npy'  ,  'Land_low_confidence_SSP585_2300_quantiles95.npy'  ,  'Land_medium_confidence_SSP585_2100_quantiles50.npy'  ,
              'Land_medium_confidence_SSP585_2100_quantiles95.npy'  ,  'Land_medium_confidence_SSP585_2300_quantiles50.npy'  ,  'Land_medium_confidence_SSP585_2300_quantiles95.npy'  ,
              'ocean_low_confidence_SSP585_2100_quantiles50.npy'  ,  'ocean_low_confidence_SSP585_2100_quantiles95.npy'  ,  'ocean_low_confidence_SSP585_2300_quantiles50.npy'  ,
              'ocean_low_confidence_SSP585_2300_quantiles95.npy'  ,  'ocean_medium_confidence_SSP585_2100_quantiles50.npy'  ,  'ocean_medium_confidence_SSP585_2100_quantiles95.npy'  ,
              'ocean_medium_confidence_SSP585_2300_quantiles50.npy'  ,  'ocean_medium_confidence_SSP585_2300_quantiles95.npy',
             'mask.npy', 'elevation.npy', 'gia_geoid.npy', 'gia_geoid_unc.npy', 'pdmt_geoid.npy', 'pdmt_geoid_unc.npy']
variables = [file_name.split('.')[0] for file_name in file_names]

# Don't mask data 💀💀💀💀 data loss danger
# instead of masking I will PLOT a Land layer above the data without messing up the data
# That will be done in the plot section

# Load the numpy arrays
for file_name, variable in zip(file_names, variables):
    data = np.load(file_name)
    exec(f"{variable} = data")

FileNotFoundError: [Errno 2] No such file or directory: 'Antarctica.npy'

In [ ]:
# let us shorten the names
# both names can be used
#tabel 9.8 IPCC

Land_Low_2100_q50  = Land_low_confidence_SSP585_2100_quantiles50
Land_Low_2100_q95  = Land_low_confidence_SSP585_2100_quantiles95
Land_Low_2300_q50  = Land_low_confidence_SSP585_2300_quantiles50
Land_Low_2300_q95  = Land_low_confidence_SSP585_2300_quantiles95

Land_Med_2100_q50 = Land_medium_confidence_SSP585_2100_quantiles50
Land_Med_2100_q95 = Land_medium_confidence_SSP585_2100_quantiles95
Land_Med_2300_q50 = Land_medium_confidence_SSP585_2300_quantiles50
Land_Med_2300_q95 = Land_medium_confidence_SSP585_2300_quantiles95

Ocean_Low_2100_q50 = ocean_low_confidence_SSP585_2100_quantiles50
Ocean_Low_2100_q95 = ocean_low_confidence_SSP585_2100_quantiles95
Ocean_Low_2300_q50 = ocean_low_confidence_SSP585_2300_quantiles50
Ocean_Low_2300_q95 = ocean_low_confidence_SSP585_2300_quantiles95

Ocean_Med_2100_q50 = ocean_medium_confidence_SSP585_2100_quantiles50
Ocean_Med_2100_q95 = ocean_medium_confidence_SSP585_2100_quantiles95
Ocean_Med_2300_q50 = ocean_medium_confidence_SSP585_2300_quantiles50
Ocean_Med_2300_q95 = ocean_medium_confidence_SSP585_2300_quantiles95

##Oelsmann Data

In [7]:
#voorbeeld inlezen en foutmelding als bestand niet gevonden wordt
url = 'https://github.com/alatieh/IPO/raw/Relative-Sea-Level-fom-NetCDF/SeaLevel.nc'
filename = 'SeaLevel.nc'

if not os.path.isfile(filename):
    urllib.request.urlretrieve(url, filename)

dataset = nc.Dataset(filename, 'r')


HTTPError: HTTP Error 404: Not Found

In [ ]:
#Retrieve data from the Oelsmann et al. (2024)'s paper
#file_path = r"C:\Users\sanne\OneDrive\Documents\Study\Master\Guided Research\Data\Oelsmann\Vertical_land_motion_preliminary_point_wise_trends.nc"

#Oelsmann_data = nc.Dataset(file_path)

##Nicholls data (VLM, 117 cities)

###Dataset
Data from Nicholls et al., 2021

In [ ]:
Nicholls_data = [["Amazon", -1.2],
                 ["Amur", 1.0],
                 ["Apalachicola", 1.0],
                 ["Arno", 1.0],
                 ["Bahia Blanca", 1.0],
                 ["Balsas", 1.0],
                 ["Baram River", 1.0],
                 ["Batang Hari", 1.0],
                 ["Betsiboke River", 1.0],
                 ["Brazos", 1.0],
                 ["Burdekin", -1.5],
                 ["Cauvery", 1.0],
                 ["Chao Phraya", 1.5],
                 ["Colorado (Gulf of California)", 6.3],
                 ["Colorado (Gulf of Mexico)", 1.0],
                 ["Copper", 1.0],
                 ["Danube", 1.6],
                 ["Daugava", 1.0],
                 ["Dneiper", 2.5],
                 ["Doce", 1.0],
                 ["Don", 1.0],
                 ["Ebro", 2.6],
                 ["Eel", 1.0],
                 ["Elwha", 1.0],
                 ["Fly River", 1.0],
                 ["Fraser", 1.0],
                 ["Ganges/Brahmapruta", 3.0],
                 ["Godavari", 7.1],
                 ["Grijalva/Usmacinto", 2.5],
                 ["Guayas", 1.0],
                 ["Han, Korea", 1.0],
                 ["Hong (Red River)", -1.2],
                 ["Indigirka", 1.0],
                 ["Indragiri River", 1.0],
                 ["Indus", 2.3],
                 ["Irrawaddy", -1.5],
                 ["Jequitinhonha", 1.0],
                 ["Kapuas", 1.0],
                 ["Kelang River", 1.0],
                 ["Klang", 1.0],
                 ["Kolyma", 1.0],
                 ["Krishna", 7.9],
                 ["Kuban", 1.0],
                 ["Lena", -1.3],
                 ["Liaohe", 1.0],
                 ["Limpopo", 1.0],
                 ["Llobregat", 1.0],
                 ["MacKenzie", 0.4],
                 ["Magdalena", 1.2],
                 ["Mahakam", -0.5],
                 ["Mahanadi", 3.6],
                 ["Mangoky", 1.0],
                 ["Mania", 1.0],
                 ["McArthur", 1.0],
                 ["Mekong", 11.0],
                 ["Mississippi", 6.0],
                 ["Mono", 1.0],
                 ["Moulouya", 2.6],
                 ["Murray", 1.0],
                 ["Nagara River", 1.0],
                 ["Neman", 1.0],
                 ["Niger", 4.4],
                 ["Nile (East)", 3.9],
                 ["Nile (West)", 0.4],
                 ["North Italian Plain", 5.0],
                 ["North Java Coastal Plain", 1.0],
                 ["Olenyok", 1.0],
                 ["Ombrone", 1.0],
                 ["Orange River", 1.0],
                 ["Ord River", 1.0],
                 ["Orinoco", 2.5],
                 ["Oueme", 1.0],
                 ["Paraiba do Sol River", 1.0],
                 ["Parana", 1.9],
                 ["Parnaiba", 1.0],
                 ["Pechora", 1.0],
                 ["Penna River", 1.0],
                 ["Pungwe River", 1.0],
                 ["Purari", 1.0],
                 ["Rhine/Scheldt/Meuse", 1.0],
                 ["Rhone", 1.0],
                 ["Rio Grande", 4.6],
                 ["Rio Negro", 1.0],
                 ["Rufiji", 1.0],
                 ["Salween", 1.0],
                 ["Sao Francisco, Brazil", 2.9],
                 ["Sebou", 0.0],
                 ["Semani", 1.0],
                 ["Senegal", 2.3],
                 ["Sepik", 1.0],
                 ["Seyhan", 1.0],
                 ["Shantou", 1.0],
                 ["Shatt el Arab", 3.6],
                 ["Shkumbin", 1.0],
                 ["Sinu", 1.0],
                 ["Taiwan east coast", 1.0],
                 ["Tana River", 1.0],
                 ["Tone-gawa", 1.0],
                 ["Tordera", 1.0],
                 ["Trinity River estuary, Galveston", 1.0],
                 ["Tumen", 1.0],
                 ["Var", 1.0],
                 ["Vistula/Wisla", 1.0],
                 ["Vjose", 1.0],
                 ["Volta", 2.5],
                 ["Waipaoa", 1.0],
                 ["Xi Jiang (Pearl)", 2.5],
                 ["Yamazaki River (Tokyo)", 1.0],
                 ["Yana", 1.0],
                 ["Yangtze", 2.1],
                 ["Yellow (Huang He)", 1.8],
                 ["Yenisey River", 1.0],
                 ["Yodo River (Osaka)", 1.0],
                 ["Yukon", -1.5],
                 ["Zaire River", 1.0],
                 ["Zambezi", 1.0],
                 ["Zarumilla River", 1.0]
]

Nicholls_data_correct = [{"city": city, "subsidence (mm/yr)": subsidence} for city, subsidence in Nicholls_data]
print(Nicholls_data_correct)

###Visualisation in table

In [ ]:
#subsidence: a positive value indicates subsidence and a negative value indicated aggradation.

headers = ["city", "subsidence (mm/yr)", "latitude", "longitude"]

print(tabulate([list(d.values()) for d in Nicholls_data_correct], headers=headers, tablefmt="fancy_grid", showindex="always"))
#print(tabulate(Nicholls_data_correct, headers=headers, tablefmt="fancy_grid", showindex="always"))

#Compare SEJ/AR6/HE

In [ ]:
# do some calculations to compare SEJ/AR6/HE
# ocean q95 on average 0.36 in 2100 q50 0.30 q95 1.51 in 2300
# land q95 on average 0.04 in 2100  q50 0.03 q95 0.1 in 2300
# base line IPCC AR6 fields 1995-2014 hence 2005 reference year
# SEJ = Structured Expert Judgement

# conversion of gia rate in mm/yr to m over a certain period
# GIA is defined positive in uplift regions hence we need to multiply with -1 for RSL
time21= 2100- 2005
time23= 2300- 2005
Gia21 = - gia_geoid * time21 / 1000
Gia23 = - gia_geoid * time23 / 1000
Gia21_unc=gia_geoid_unc * time21 / 1000
Gia23_unc=gia_geoid_unc * time23 / 1000

#convert to high-end -> the GIA plus the uncertainty
Gia21HE = Gia21 + 2 * Gia21_unc
Gia23HE = Gia23 + 2 * Gia23_unc

#convert to high-end 2100
#AR6 numbers from table 9.8 (SEJ)
allHE_5C_2100 =     Greenland*0.29 + Antarctica*0.59 + Glaciers*0.27 + Ocean_Low_2100_q95 + Land_Low_2100_q95 + Gia21HE
allHE_5C_2100_rel = allHE_5C_2100 /1.55
allAR6_5C_2100 =     Greenland*0.59 + Antarctica*0.56 + Glaciers*0.20 + Ocean_Low_2100_q95 + Land_Low_2100_q95 + Gia21HE
allAR6_5C_2100_rel = allAR6_5C_2100/1.75

#combine them plot9-12
#AR6 numbers from table 9.11 (SEJ, MICI -> highest numbers)
allHE_9C_2300 =      Greenland*2.5 + Antarctica*6 + Glaciers*0.32 + Ocean_Low_2300_q95 + Land_Low_2300_q95 + Gia23HE
allHE_9C_2300_rel = allHE_9C_2300/10.43
allAR6_9C_2300 =     Greenland*2.23 + Antarctica*13.54 + Glaciers*0.32 + Ocean_Low_2300_q95 + Land_Low_2300_q95 + Gia23HE
allAR6_9C_2300_rel = allAR6_9C_2300/16.2

#combine them plot13-14
#median from table 9.8 (not SEJ)
allAR6_5C_2100_med =     Greenland*0.13 + Antarctica*0.12 + Glaciers*0.18 + Ocean_Low_2100_q50 + Land_Low_2100_q50 + Gia21HE
allAR6_5C_2100_med_rel = allAR6_5C_2100_med/0.77

#combine them plot15-16
#95% percentile SEJ and 1783 perc for the rest -> not sure where these numbers come from
allAR6_5C_2100_95perc =     Greenland*0.98 + Antarctica*1.31 + Glaciers*0.20 + Ocean_Low_2100_q95 + Land_Low_2100_q95 + Gia21HE
allAR6_5C_2100_95perc_rel = allAR6_5C_2100_95perc/2.89

#Define functions
for easy slicing and plotting

In [ ]:
#the following codes are not included here:
# 1. extract array name
# 2. percentile between 10 and 90 (something with colors)
# 3. definition removes NaN values

In [ ]:
# make a function that extract the array name
def get_array_name(array):
    for name, value in globals().items():
        if value is array:
            return name
    return None

In [ ]:
# I want the color to begin in the percentile 10 and end in 90 to avoid outliers and improve the pattern
# The following function automat finding the wanted_percentile value
"""def perc(data, wanted_percentile):
    unique_values = np.unique(data[~np.isnan(data)])
    percentile = np.percentile(unique_values, wanted_percentile)
    return percentile"""

In [ ]:
def best_bins(array, x):
    """
    This function takes an array,removes the NaN, flatten it, arrange it
    then it and divides it into x parts,
    then returns the average of each part.
    This function is handy to select the best contour lines to delineate

    Parameters:
    array (numpy.ndarray): The input array. You could use any shape, for example 2D or 3D
    x (int): The number of parts to divide the array into.

    Returns:
    numpy.ndarray: An array containing the average of each part.
    """
    # Create a new array without NaN values
    no_nan_array = array[~np.isnan(array)]

    # Sort data and divide it into x parts
    parts = np.array_split(np.sort(no_nan_array), x)

    # Calculate the average of each part
    average_values = np.array([part.mean() for part in parts])

    return average_values

In [ ]:
golden_ratio = (1 + 5 ** 0.5) / 2  # calculate the golden ratio
width = 10  # set the width of the figure, in inches
height = width / golden_ratio  # calculate the height of the figure based on the golden ratio 😁

In [ ]:
# Create numpy array for longitude and latitude
lon = np.concatenate((np.arange(0, 181), np.arange(-179, 0)))
lat = np.arange(-89.5, 90, 1)

# to plot contour lines you need a continuous longitude, I will call it index_x
index_x = np.arange(0,360)
index_y = np.arange(89.5, -90, -1)

#Slice by latitude, longitude values

In [ ]:
Ocean_Med_2300_q95.shape

All are not masked now

All data in the same numpy array shape of (180, 360)   
To access a point value use slicing of (latitide, longitude)

Using longitude, use the longitude value between 0 and 359  
So if you use a negative longitude then you need to take 360+the longitude  

To access the value using latitude, plug in the value of (`90-latitude`)   
This can be used for both (negative latitude in Southern hemisphere) or ( positive in Southern hemisphere)  
Correction: our latitude grid is every half degree so that become (`89.5-latitude`)   

Let us make a function that slice by takeing the real value latitude, longitude. That will the function take the negative value also. In the next line I will work step by step and verify my work


In [ ]:
def convert_to_image_coordinate(latitude, longitude):
    # the input will be the real value latitude, longitude
    # The output will be the index in the image, the index can be read in the FastPlot
    lat_index = 89.5 - latitude
    if longitude < 0:
        lon_index = 360 + longitude
    else:
        lon_index = longitude
    return lat_index, lon_index

Some examples in the other document

In [ ]:
def easy_slice(data, latitude, longitude):
    """
    Returns the value in the data array that is closest to the given latitude and longitude.

    Parameters:
    data (ndarray): The input data array.
    latitude (float): The latitude value. Enter the coordinates as decimal
    longitude (float): The longitude value. Enter the coordinates as decimal

    Returns:
    float: The value in the data array that is closest to the given latitude and longitude.
    """
    lat_index, lon_index = convert_to_image_coordinate(latitude, longitude)
    value = data[int(lat_index), int(lon_index)]
    return value

#Slice by city

##Tools for coordinates per city/river

In [ ]:
# I don't want to search for the city coordinates in google 😖
# I will use geopy database with Nominatim for this task

def get_coordinates(city_name):
    geolocator = Nominatim(user_agent="MyCityMApp")
    location_city = geolocator.geocode(city_name)

    if location_city is not None:
        latitude, longitude = location_city.latitude, location_city.longitude
        return latitude, longitude
    else:
        return None

In [ ]:
#Retrieve the coordinates for all cities from the dataset

def get_coordinates_with_retry(city_name):
    base_url = "https://nominatim.openstreetmap.org/search"
    params = {
        'q': city_name,
        'format': 'json',
        'limit': 1
    }

    session = requests.Session()
    retry_strategy = Retry(
        total=3,
        backoff_factor=1,
        status_forcelist=[500, 502, 503, 504],
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("https://", adapter)

    try:
        response = session.get(base_url, params=params, timeout=5)
        response.raise_for_status()
        data = response.json()
        if data:
            location = data[0]
            latitude, longitude = float(location['lat']), float(location['lon'])
            return latitude, longitude
    except requests.exceptions.RequestException as e:
        print(f"Error while getting coordinates for {city_name}: {e}")

    return None

In [ ]:
def find_city_with_retry(dataset):

    """ find coordinates for the cities from the dataset
        input: dataset with city names in the first column
        output: coordinates (latitude and longitudes) from the cities from the dataset
    """

    coordinates_list = []

    for city_entry in dataset:
        city_name = city_entry["city"]
        coordinates = get_coordinates_with_retry(city_name)

        if coordinates is not None:
            city_entry["latitude"] = coordinates[0]
            city_entry["longitude"] = coordinates[1]
            coordinates_list.append(city_entry)
        else:
            # Handle the case where coordinates are not found for a city
            city_entry["latitude"] = np.nan
            city_entry["longitude"] = np.nan
            print(f"Coordinates not found for {city_name}")

    return coordinates_list

In [ ]:
coordinates_Nicholls_data = find_city_with_retry(Nicholls_data_correct)

In [ ]:
#for entry in coordinates_Nicholls_data:
 #   print(f"City: {entry['city']}, Latitude: {entry['latitude']}, Longitude: {entry['longitude']}")

In [ ]:
#find coordinates for the cities
#sad part is that many are rivers, so I either need to find the closest cities or find another tool to find the coordinates!
#def find_city(dataset):
 #   for city_entry in dataset:
  #      city_name = city_entry["city"]  # Assuming city name is at index 0
   #     coordinates = get_coordinates(city_name)

    #    if coordinates is not None:
     #       city_entry["latitude"] = (coordinates[0])
      #      city_entry["longitude"] = (coordinates[1])
       # else:
            # Handle the case where coordinates are not found for a city
        #    print(f"Coordinates not found for {city_name}")
#find_city(Nicholls_data_correct[20:40])

In [ ]:
#col_names = ["city", "subsidence (mm/yr)"]

#print(tabulate([list(d.values()) for d in Nicholls_data_correct], headers=headers, tablefmt="fancy_grid", showindex="always"))
#print(tabulate(Nicholls_data, headers=col_names, tablefmt="fancy_grid", showindex="always"))

In [ ]:
def city_slice(data, cities):
    """
    Returns the value in the data array that is closest to the given city.

    Parameters:
    data (ndarray): The input data array.
    city name. Enter the city as text between ""

    Returns:
    float: The value in the data array that is closest to the given latitude and longitude.
    """

    values = {}
    for city in cities:
        coordinates = get_coordinates_with_retry(city)
        if coordinates is not None:
            lat_index, lon_index = convert_to_image_coordinate(coordinates(city)[0], coordinates(city)[1])
            value = data[int(lat_index), int(lon_index)]
            values[city] = value
        else:
            print(f"Coordinates not found for {city}")

    return values

In [ ]:
Nicholls_slice = city_slice(allAR6_5C_2100_95perc, [entry["city"] for entry in Nicholls_data_correct])
#Nicholls_slice

In [ ]:
Nicholls_slice_correct = [{"city": city, "subsidence (mm/yr)": subsidence} for city, subsidence in Nicholls_data]
#Nicholls_slice_correct

##Extract values for specific cities

In [ ]:
def All_for_city(city_entry):
    """
    Function to retrieve sea level rise data for a specific city.

    Parameters:
    city (str): The name of the city.

    Returns:
    The sea level according to the different scenario's in neat table orderd from low to highest scenario
    """

    coordinates = find_city_with_retry(city_entry)
    if coordinates is not None:
       x, y = coordinates  # Extract latitude and longitude
    else:
       return None

    datasets_names = ['allHE_5C_2100', 'allHE_5C_2100_rel', 'allAR6_5C_2100', 'allAR6_5C_2100_rel', 'allHE_9C_2300', 'allHE_9C_2300_rel', 'allAR6_9C_2300', 'allAR6_9C_2300_rel', 'allAR6_5C_2100_med', 'allAR6_5C_2100_med_rel', 'allAR6_5C_2100_95perc', 'allAR6_5C_2100_95perc_rel']

    datasets_set = [allHE_5C_2100, allHE_5C_2100_rel, allAR6_5C_2100, allAR6_5C_2100_rel, allHE_9C_2300, allHE_9C_2300_rel, allAR6_9C_2300, allAR6_9C_2300_rel, allAR6_5C_2100_med, allAR6_5C_2100_med_rel, allAR6_5C_2100_95perc, allAR6_5C_2100_95perc_rel]

    table = []
    for dataset, dataset_name in zip(datasets_set, datasets_names):
        dataset_sliced = round(easy_slice(dataset, y, x), 2)
        table.append([dataset_name, dataset_sliced])

    table.sort(key=lambda x: x[1])

    print(tabulate(table, headers=["scenario", "Sea Level Rise (m)"]))


#Visualisation maps for cities

In [ ]:
def CartopyPlot(data1, data2, c=None, projection=None, city=None):

    """
    Function to create a Cartopy plot with various customization options.

    Parameters:
    The only needed parameter is the data name, the other parameters are optional
    - data: numpy array, the data to be plotted
    - c: str, optional, color scale type. 'd' for discrete color scale, any other value for continuous color scale  (default)
    - projection: str, optional, type of map projection. 'eu' for EuroPP projection, 'flat' for PlateCarree projection, and None for Robinson projection (default)
    - city: str or list of str, optional, cities to be marked on the map. Default is "Den Haag"

    Returns
    nice plot
    """

    fig = plt.figure(figsize=(width, height))

    levels = best_bins((data), 9)
    cmap = plt.get_cmap('viridis') # The color map that we will use

    # choose projection
    if projection is None:
        projection = ccrs.Robinson() # The default projection
    elif projection == 'eu':
        projection = ccrs.EuroPP()
        eu_data= np.concatenate((data1[10:60, 350:360],data1[10:60, 0:35]), axis=1 )
        levels = best_bins((eu_data), 9)
        fig = plt.figure(figsize=(16, 12))
    elif projection == 'flat':
        projection = ccrs.PlateCarree()

    ax = fig.add_axes([0, 0, 1, 1], projection = projection)

    # continuous or discrete color scale?
    if c is None or c != 'd':
        norm = colors.Normalize(vmin=levels[0], vmax=levels[-1])  # default, This will make continuous normalized color scale
    else:
        norm = colors.BoundaryNorm(levels, cmap.N) # use discrete colors scale

    mesh = ax.pcolormesh(lon, lat, np.flipud(data), cmap=cmap, norm=norm, transform=ccrs.PlateCarree())
    cbar = plt.colorbar(mesh, ax=ax, orientation='horizontal', shrink=0.6, pad=0.05)
    cbar.set_label('meter')

    cs = plt.contour(index_x, index_y, data1, levels=levels, colors='gray', transform=ccrs.PlateCarree(), linestyles='dashed')
    plt.clabel(cs, inline=True, fontsize=10, fmt='%1.1f', colors='black')

    ax.add_feature(cfeature.LAND, edgecolor='k', zorder=4)
    ax.add_feature(cfeature.RIVERS, zorder=4)
    ax.add_feature(cfeature.LAKES, zorder=4)

    ax.stock_img() # not working

    ax.set_title(get_array_name(data1))

    #cities to the map with different colors
    for entry in data2:
        x = entry['longitude']
        y = entry['latitude']

        if entry['subsidence (mm/yr)'] <= Nicholls_slice_correct['subsidence (mm/yr)']:
            a='bo'
        else:
            a='ro'

        ax.plot(x, y, a, markersize=7, transform=ccrs.PlateCarree(), zorder=4)


In [ ]:
#coordinates_Nicholls_data
print(type(coordinates_Nicholls_data[1]))
print(type(Nicholls_slice_correct['subsidence (mm/yr)'][::]))

In [ ]:
#CartopyPlot(Greenland)
CartopyPlot(allAR6_5C_2100_95perc, Nicholls_data_correct, c='c',projection = None,
            city= coordinates_Nicholls_data[0])